In [ ]:
import os
import boto3
from os.path import join
from anndata import read_h5ad
import scanpy as sc

from vitessce import (
    VitessceWidget,
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
    upload_to_s3,
)

In [ ]:
s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=os.environ['VITESSCE_S3_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['VITESSCE_S3_SECRET_ACCESS_KEY'],
)

In [ ]:
adata = read_h5ad(join("data", "habib17.processed.h5ad"))
sc.pp.highly_variable_genes(adata, n_top_genes=100)

vc = VitessceConfig(name='Habib et al', description='COVID-19 Healthy Donor Brain')
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(adata))
scatterplot = vc.add_view(dataset, cm.SCATTERPLOT, mapping="X_umap")
cell_sets = vc.add_view(dataset, cm.CELL_SETS)
genes = vc.add_view(dataset, cm.GENES)
heatmap = vc.add_view(dataset, cm.HEATMAP)
vc.layout((scatterplot | (cell_sets / genes)) / heatmap);

In [ ]:
upload_to_s3(vc, s3, 'vitessce-export-examples', prefix='test')